# Symmetric Key Encryption

Symmetric keys 演算法可分為: stream 和 block cipher。

- Stream ciphers 作用在data stream上， 也就是一次1 bit 或 1 byte
- Block ciphers 作用在 blocks of data， 例如一次 16 bytes 


1. Blcok Cipher

1.1 AES

1.2 Utility
    
    1.2.1 Padding
    
    1.2.2 Cipher Mode
    
    1.2.3 Random number

1.3 Operation Flow

1.4 Full AES Example

1.5 The Recipe Layer

2. Stream cipher

2.1 Nacl

2.2 Final notes

## Block Cipher
我們介紹會以AES為主，不過相關的概念皆通用所有的block cipher。基本上我們所講的加解密絕大多數都是指對稱式加解密，簡單來說，就是用同一把 secret key 進行Encrypt和Decrypt的動作。優點是對於資料大的檔案加解密的速度較快，而且容易透過軟硬體實作，運算所需要的記憶體較少，但是缺點是在資料交換的過程中，雙方必須取得這把相同的secret key。否則，接收到加密檔案的接收者，無法解開加密後的檔案。

> 我們每天都會用到它，像大多數網站http -> https就是加上了加解密服務。

### AES
最廣為使用的 symmetric cipher 和 standard 是 the AES block cipher, 見 [FIPS PUB 197](http://csrc.nist.gov/publications/fips/fips197) 

AES(Advanced Encryption Standard)是透過對每個固定大小的4x4位元矩陣區塊(block = 128 bits = 16 bytes)，對其每一個元素進行多次交互置換(permutation)和取代(substitution)運算(S-P network)。而它的安全性強度取決於金鑰(secret key)的長度。

> 對稱式加解密理論上是為了達成兩個原則，Confusion 表示 ciphertext 的每一個bit要盡量和key的許多bit有關。Diffusion表示改變一個plaintext的bit需要讓近乎一半的ciphertext有關。雖然不像非對稱式有完整的數論基礎，不過這兩個法則已被研究許久，一般相信是沒有問題的!

> 另一種常見的架構是Feistel Cipher，他在設計上模組化，加解密都用同一套硬體即可，也被很多cipher採用

目前美國國防安全局審核認定的AES標準演算法secret key長度有128 bits, 192 bits和256 bits 三種。secret key的長度越長就越安全，但是相對的加密所需的時間就越多，然而實際上secret key長度對於加密時間的影響並不大(軟硬體實作上)，主要還是取決於所選擇的 Block Cipher Mode以及需要加密的資料大小。

> keylength estimation https://www.keylength.com/ 可以找到安全度的建議，基本上對稱式金鑰不存在比暴力攻擊更快的方法，引此他的金鑰長度就等於安全度，相對來說非對稱式金鑰就要看目前研究中最快的攻擊法需要的複雜度是多少

建議可先看[這個](http://www.formaestudio.com/rijndaelinspector/archivos/Rijndael_Animation_v4_eng.swf) flash動畫來複習AES!

在網路上的文章或是API文件中，常見到以下這些名詞：

## Utility
### 1) Padding
由於加密過程，是針對每個固定大小的區塊(16 bytes)，進行多次的運算，因此當需要被加密的資料小於矩陣區塊16 bytes 的時候，或是資料的size 不是 16 bytes的倍數時，為了讓加密能夠順利進行，必須將資料的 size 補齊到能夠被 16 bytes 整除的大小，也就是我們需要放寬第一章所說的byte長度限制的問題

> AES 為 Rijndael cipher的一部分. AES 以 128-bits (16 bytes) 固定為block size; 而data必須經過padding來滿足block size的倍數.例如 "ABCDABCDABCDABCDABCDABCDABCDABCD" 這樣的訊息不用做padding。 但若訊息是 "ABCDABCDABCDABCDABCDABCDABCD" 則需要 4 byte的 padding

> 一個常見的padding 方式為將 0x80 當作第一個padding的byte，然後用 0x00 填滿剩下的padding。 例如前述例子會變成:"ABCDABCDABCDABCDABCDABCDABCD\x80\x00\x00\x00"

我們先定義padding和unpadding的函式

In [1]:
def pad_data(data):
    """pad_data pads out the data to an AES block length."""
    # return data if no padding is required
    if len(data) % 16 == 0:
        return data
    # subtract one byte that should be the 0x80 if 0 bytes of padding are required, it means only a single \x80 is required.
    padding_required = 15 - (len(data) % 16)
    data = '%s\x80' % data
    data = '%s%s' % (data, '\x00' * padding_required)
    return data

def unpad_data(data):
    """unpad_data removes padding from the data."""
    if not data:
        return data
    data = data.rstrip('\x00')
    if data[-1] == '\x80':
        return data[:-1]
    else:
        return data

> 除了padding外另一種常見的處理方式為cipher stealing

### 2) Cipher mode
實務上用block cipher加密不會只加密一個block，所以我們必須將資料切塊後做padding來進行加密。最直覺的方法為對每一個區塊進行AES加密，也就是**electronic codebook (ECB)** mode，但是這種方法會有**同一資料總是對應通一加密結果的問題**。故一般會推薦使用其他像 cipher block chaining **CBC mode** 或 counter **CTR mode** (See Cryptography Engineering by Bruce Schneier). 

> 還有其他像**cipher feedback (CFB)**， **output feedback (OFB)**。

#### CBC
CBC mode 是被廣泛使用的一種mode，我們會以他為主來探討。進一步來說 Cipher block chaining 會去 XORing 前一個 ciphertext block 和目前的 block。而一開始的bootstrap機制是用 **initialization vector (IV)**來達成。這樣就可以確保加密多次的情況下同一plaintext不會對應相同的ciphertext。

![](https://i.imgur.com/dF9LU1z.png)

> IV 的條件為一連串的randomly-generated bytes，並且要夠random無法反推!! 另外他是可以公開的。也就是說加解密實際上可以看成機密性的壓縮，我們將原來要保護的長串文章，轉換成只要保護key即可!!

> 加解密最重要的部分之一是正確的產生 random data。這裡我們採用 PyCrypto library’s 中的 _Crypto.Random.OSRNG module_. 記住當我們有更多的 entropy sources (例如 network traffic 和 disk activity)，為的就是讓系統能更快速的產生 cryptographically-secure random data，這也就是為什麼一些加解密軟體一開始會要使用者隨意移動滑鼠的原因。下面我們寫一個適用於linux系統產生 initialization vector 的function(從/dev/uramdom 不是 /dev/random。 注意 GMP的 library 可以更新到  5.0 (timing attack resistant) 或 6.0 (side channel silence mode) 來對抗 physical attacks。

### 3) Randome number

In [4]:
import Crypto.Random.OSRNG.posix as RNG
import Crypto.Cipher.AES as AES 
import os
def generate_nonce():
    """Generate a random number used once."""
    #return RNG.new().read(AES.block_size)
    return os.urandom(AES.block_size)

In [5]:
nonce =  generate_nonce()
nonce, type(nonce)

('/\x1aQ|\xad?21\xb881L\x07\xfc\xaa\xaf', str)

> 注意 python 內建的 random module 為MT19937 是完全可預測的。 並不適合拿來寫 cryptographic code

## Operation Flow

### (1) Key generation

AES 有三種key size: **128-bit, 192-bit, and 256-bit, 或計做 16-byte, 24-byte, and 32-byte key sizes**。 以下函數可以產生隨機的32 byte並且保存他。

In [10]:
KEY_SIZE = 32
def generate_key():
     return os.urandom(KEY_SIZE)

In [11]:
key =  generate_key()
key, type(key)

('\x08W\x9e\xbb@O\x07f\xb3\x19\xa4x%j\xf9t\xf3M\xdf\x19#}\x82i1B8\xd2\x0b\x87\xe0\x92',
 str)

### (2) Encryption/Decryption

我們可以用這把key來加解密資料。加密的話我們需要 initialization vector (i.e. a nonce), key, 和 data(plaintext)。注意 **IV 是可以公開的**，為了讓接收方能夠順利的解密我們需要某種機制來讓對方知道。以下當我們加密時IV會附加在 encrypted data 前。注意安全上考量，我們必須要對每一個新的plaintext產生不同新的random IV。 

In [12]:
def encrypt(data, key):
     """
     Encrypt data using AES in CBC mode. The IV is prepended to the ciphertext.
     """
     data = pad_data(data)
     ivec = generate_nonce()
     aes = AES.new(key, AES.MODE_CBC, ivec)
     ctxt = aes.encrypt(data)
     return ivec + ctxt # 16 byte IV + 16 byte 
 
def decrypt(ciphertext, key):
     """
     Decrypt a ciphertext encrypted with AES in CBC mode; assumes the IV has been prepended to the ciphertext.
     """
     if len(ciphertext) <= AES.block_size:
         raise Exception("Invalid ciphertext.")
     ivec = ciphertext[:AES.block_size]
     ciphertext = ciphertext[AES.block_size:]
     aes = AES.new(key, AES.MODE_CBC, ivec)
     data = aes.decrypt(ciphertext)
     return unpad_data(data)

In [19]:
key = generate_key()
plaintext = 'Martinet!'
print "The key size is: %d, block size is %d" % (len(key), AES.block_size)
print "plaintext is: %s, and the length of plaintext is %d " % (plaintext, len(plaintext))
ciphertext = encrypt(plaintext, key)
print "ciphertext is: %s, and the length of ciphertext is %d " % (ciphertext, len(ciphertext)) # 印出好看用!
print ciphertext.encode('hex')
plaintext = decrypt(ciphertext, key)
# Resulting plaintext
print plaintext

The key size is: 32, block size is 16
plaintext is: Martinet!, and the length of plaintext is 9 
ciphertext is: ��2�et�<j�cd�H̖�Yꈑ�#YF(�zq�, and the length of ciphertext is 32 
c1c032b86574e23c136a846364a748cc96e2591cea8891c523594628927a71c9
Martinet!


### (3) 加上 authentication

不過要注意的是 AES 僅確保了 confidentiality， 我們常常還需要 **integrity 和 authenticity** 等性質。(或許直覺會想到MD5或SHA，不過他們缺點是digest可以被隨意改變(且MD5已不再安全))。這時我們需要one-way function也就是hash函數

> Authenticated Encryption (AE) 或 Authenticated Encryption with Associated Data (AEAD)，事實上大多的安全專家都會推薦使用這種方法，因為沒有integrity和authenticate的加解密很容易被攻擊。

這邊我們需要的是將 hash function 加入 key 來產生 digest。也就是我們需要 HMAC (這是一個 general 的 framework)。另外，實務上我們不會用跟加密一樣的key。 我們需要的是 **新產生的另一把 key**。

> 也有人稱HMAC為symmetric signatures來對照數位簽章

這個framework中，我們需要定義使用的Hash函數。因為之前我們使用 AES-256， 所以以下函式使用 SHA-512 (注意因為 birthday attack長度須為兩倍!!)。 

> WPA2, TLSv1弱點在於hash用sha1等較早的standard

我們的 message tags 是經由 HMAC-SHA-512 來計算。 將產生 64-byte digest。 我們以下定義一些constant；

In [20]:
__AES_KEYLEN = 32

In [21]:
import Crypto.Hash.HMAC as HMAC
import Crypto.Hash.SHA512 as SHA512
 
def new_tag(ciphertext, key):
     """Compute a new message tag using HMAC-SHA-512."""
     return HMAC.new(key, msg=ciphertext, digestmod=SHA512).digest()

這是我們修改後的加密函式!

In [22]:
def encrypt(data, key):
    """
    Encrypt data using AES in CBC mode. The IV is prepended to the ciphertext.
    """
    data = pad_data(data)
    ivec = generate_nonce()
    aes = AES.new(key[:__AES_KEYLEN], AES.MODE_CBC, ivec)
    ctxt = aes.encrypt(data)
    tag = new_tag(ivec + ctxt, key[__AES_KEYLEN:]) 
    return ivec + ctxt + tag

#### Timing attack

解密做的事是確認我們的 message tag 必須正確。 然而一般來說直接使用 ``==`` 運算子會循序進行比對每一個byte並在第一次不對的時候就跳出(純速度考量)。 這樣的比對容易被 **timing attack** 所攻擊。在python中我們可以用 streql package (i.e. pip install streql) 來進行 constant-time 的比對

In [23]:
import streql
def verify_tag(ciphertext, key):
    """Verify the tag on a ciphertext."""
    tag_start = len(ciphertext) - __TAG_LEN
    data = ciphertext[:tag_start]
    tag = ciphertext[tag_start:]
    actual_tag = new_tag(data, key)
    return streql.equals(actual_tag, tag)

ImportError: No module named streql

In [ ]:
def decrypt(ciphertext, key):
     """
     Decrypt a ciphertext encrypted with AES in CBC mode; assumes the IV has been prepended to the ciphertext.
     """
     if len(ciphertext) <= AES.block_size:
         return None, False
     tag_start = len(ciphertext) - __TAG_LEN
     ivec = ciphertext[:AES.block_size]
     data = ciphertext[AES.block_size:tag_start]
     if not verify_tag(ciphertext, key[__AES_KEYLEN:]):
         return None, False
     aes = AES.new(key[:__AES_KEYLEN], AES.MODE_CBC, ivec)
     data = aes.decrypt(data)
     return unpad_data(data), True

In [ ]:
key = generate_key()
plaintext = 'Martinet!'
print len(key), len(plaintext)
ciphertext = encrypt(plaintext, key)
print len(ciphertext), ciphertext, ciphertext.encode('hex') # 印出好看用!

In [ ]:
plaintext = decrypt(ciphertext, key)
# Resulting plaintext
print plaintext

> 另外還有其他值得注意的模式像是 GCM，CCM， XTS(Cipher Stealing的技巧，常用於disk加密)


### (4) Key Derivation Function

前述我們是使用隨機產生的金鑰，在現實生活中我比較常見們比較常見的做法是讓使用者輸入password然後將其經過處理轉成金鑰。有什麼方法能從password產生出key呢? 沒錯...，熟悉的又來了，我們需要一個one-way function也就是hash，不過並不建議用上述的hash。我們通常會用另一系列的hash來建構所謂 key derivation function (KDF)，像是PBKDF2。 這類function通常還會需要 salt來增加安全度。因此 PBKDf2 有三個輸入；passphrase， salt， 和hash執行的次數。 目前普遍認為安全的 iterations 為 16384。

> Salt 是隨機產生的數，他可以確保KDF在相同 passphrase 下有不同結果。 通常來說他的長度要求為 16 bytes (128-bits)。

> 實務上建議用之順序 1)scrypt， 2)bcrypt，3) PBKDF2.

接下來我們實做兩個函數： generate a random salt 和 generate a secret key from PBKDF2 (WPA2採用):

In [ ]:
import pbkdf2 
KEYSIZE = 32
def generate_salt(salt_len):  
    """Generate a salt for use with PBKDF2."""
    return RNG.new().read(salt_len)

def password_key(passphrase, salt=None):
    """Generate a key from a passphrase. Returns the tuple (salt, key)."""
    if salt is None:
        salt = generate_salt(16)
    passkey = pbkdf2.PBKDF2(passphrase, salt, iterations=16384).read(KEYSIZE)
    return salt, passkey

In [ ]:
keyin = 'star'
salt, key = password_key(keyin)
print (salt, key)

注意salt也是公開的，他也必須存下來來還原key，可以用同樣的方法例如頭 len(salt) bytes 附加在 ciphertext前面。

### ASCII-Armouring

有些人會習慣用base64 encode來處理， 而不是用binary data。接下來試一個完整的範例。

In [ ]:
keyin = 'star'
plaintext = 'AG is god'
salt, key = password_key(keyin)
ciphertext = encrypt(plaintext, key)

In [ ]:
print decrypt(ciphertext, key)

## Yet another AES example

下面我們舉加密圖片來看看各種Block cipher 各種mode的範例。

In [ ]:
import sys
import os
import math
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from Crypto.Cipher import AES

In [ ]:
IV_SIZE = 16
BLOCK_SIZE = 16

In [ ]:
def fn_entropy(s):
    p, lns = Counter(s), float(len(s))
    return -sum( count/lns * math.log(count/lns, 2) for count in p.values())

In [ ]:
def encrypt_image(cipher_mode):
    """Encrypt an image file and write out the results as a JPEG."""
    input_image = Image.open(os.getcwd() + '/yuwen.jpg')
    print input_image
    plt.figure(figsize=(16,6))
    plt.subplot(121)
    plt.imshow(input_image)
    # Key must be one of 16/24/32 bytes in length.
    key = "0123456789ABCDEF"
    if cipher_mode == 'ECB':
        mode = AES.MODE_ECB
    elif cipher_mode == 'CBC':
        mode = AES.MODE_CBC
    elif cipher_mode == 'CFB':
        mode = AES.MODE_CFB
    elif cipher_mode == 'OFB':
        mode = AES.MODE_OFB
    iv = os.urandom(IV_SIZE)
    aes = AES.new(key, mode, iv)
    image_string = input_image.tobytes()
    # The input string must be padded to the input block size.
    image_padding_length = BLOCK_SIZE - len(image_string) % BLOCK_SIZE
    image_string += image_padding_length * "~"
    # generate the encrypted image string
    encrypted = aes.encrypt(image_string)
    # create an image from the encrypted string
    encrypted_img = Image.frombuffer("RGB", input_image.size, encrypted, 'raw', "RGB", 0, 1)
    
    plt.subplot(122)
    plt.imshow(encrypted_img)
    plt.show()
    
    # create and save the output image
    # encrypted_img.save(output_filename, 'PNG')

    print("Encrypted using AES in " + cipher_mode + " mode!")
    print 'Entropy on original:  ', fn_entropy(image_string)
    print 'Entropy on encryption: ', fn_entropy(encrypted_img.tobytes())

In [ ]:
def encrypt_text(text,cipher_mode):
    
    # Key must be one of 16/24/32 bytes in length.
    key = "0123456789ABCDEF"
    if cipher_mode == 'ECB':
        mode = AES.MODE_ECB
    elif cipher_mode == 'CBC':
        mode = AES.MODE_CBC
    elif cipher_mode == 'CFB':
        mode = AES.MODE_CFB
    elif cipher_mode == 'OFB':
        mode = AES.MODE_OFB
    iv = os.urandom(IV_SIZE)

    aes = AES.new(key, mode, iv)
    
    # Padding
    # The input string must be padded to the input block size.
    text_padding_length = BLOCK_SIZE - len(text) % BLOCK_SIZE
    text += text_padding_length * "~"
    text += text
    
    text_enc = aes.encrypt(text)
    
    print("Encrypted using AES in " + cipher_mode + " mode!")
    print 'Input text:     ', text
    print 'Encrypted text: ', text_enc.encode('hex')
    ####

In [ ]:
encrypt_text('Hello World','ECB')

In [ ]:
encrypt_text('Hello World','CBC')

In [ ]:
encrypt_image('ECB')

上圖 (Mon god) 有加密嗎?

In [ ]:
encrypt_image('CBC')

In [ ]:
encrypt_image('CFB')

In [ ]:
encrypt_image('OFB')

## Cryptography
我們用 _cryptography toolkit_來看看一個新的mode - GCM (包含加解密和認證)

In [ ]:
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

def encrypt(key, plaintext, associated_data):
    # Generate a random 96-bit IV.
    iv = os.urandom(12)

    # Construct an AES-GCM Cipher object with the given key and a
    # randomly generated IV.
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
        backend=default_backend()
    ).encryptor()

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)

    # Encrypt the plaintext and get the associated ciphertext.
    # GCM does not require padding.
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    return (iv, ciphertext, encryptor.tag)

def decrypt(key, associated_data, iv, ciphertext, tag):
    # Construct a Cipher object, with the key, iv, and additionally the
    # GCM tag used for authenticating the message.
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
        backend=default_backend()
    ).decryptor()

    # We put associated_data back in or the tag will fail to verify
    # when we finalize the decryptor.
    decryptor.authenticate_additional_data(associated_data)

    # Decryption gets us the authenticated plaintext.
    # If the tag does not match an InvalidTag exception will be raised.
    return decryptor.update(ciphertext) + decryptor.finalize()

In [ ]:
iv, ciphertext, tag = encrypt(key,b"a secret message!",b"authenticated but not encrypted payload")

print(decrypt(key,b"authenticated but not encrypted payload",iv,ciphertext,tag))

## The Recipe Layer

上述要實現一個安全的演算法所要考量的事情很多，過去十幾年來誤用的例子太多了。因此有些近期個library會統一提供單一介面讓設計者能夠更加安全的使用，當然缺點就是失去一些選擇的彈性(演算法，padding方式、安全速度tradeoff等)。**不過我們還是強烈推薦用這種方法**。

Cyrptography的統一介面叫 Fernet，他建構在一系列 standard cryptographic primitives上 包含:

- AES in CBC mode， 128-bit key， 用 PKCS7 padding。
- HMAC 用 SHA256 來認證。
- Initialization vectors 用 os.urandom() (而非語言內建)來產生。

> 大多專家會建議使用 operating system 的 random number generator，在python可透過 os.urandom()存取。在UNIX系統中他會拿 /dev/urandom 在windows會用 CryptGenRandom


因此我們要注意的只剩key的產生: 	
```
key (bytes) – A URL-safe base64-encoded 32-byte key. This must be kept secret. Anyone with this key is able to create and read messages.
```

In [ ]:
import base64
import os
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
password = b"password"
salt = os.urandom(16)
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
key = base64.urlsafe_b64encode(kdf.derive(password))
f = Fernet(key)
token = f.encrypt(b"Secret message!")
token

In [ ]:
f.decrypt(token)

## Stream Cipher

對於IoT應用來說，我們可以使用pynacl來使用Salsa20這類由eStream選出來的cipher，也是最有潛力取代AES的方案。

> http://cr.yp.to/streamciphers/why.html
> https://gist.github.com/tqbf/be58d2d39690c3b366ad
> http://cr.yp.to/ecdh.html#papers

其中Nacl/Libsodium 是一個新的加解密函式庫，他推薦的加解密方法順序是，1) The Nacl/libsodium default (Recipe layer), (2) Chacha20-Poly1305 (salsa20的版本加上認證), (3) AES-GCM (業界傳統的標準)

> Cryptography也是要提供一個python的類似介面，設計目標一樣不過架構不同。Nacl的作者為實戰理論兼具的Daniel J. Bernstein，Michael Naehrig 等人。他們的對稱式金鑰統一介面為secrtetbox，就像保險櫃一樣，你可以放東西進去只有金鑰才可以打開，而且任何的竄改都會被偵測到。而公鑰介面為box。他有tarfet IoT 的hardware的implementation https://cryptojedi.org/papers/naclhw-20150616.pdf


In [ ]:
import nacl
nacl

In [ ]:
import nacl.secret
import nacl.utils

# This must be kept secret, this is the combination to your safe
key = nacl.utils.random(nacl.secret.SecretBox.KEY_SIZE)

# This is your safe, you can use it to encrypt or decrypt messages
box = nacl.secret.SecretBox(key)

# This is our message to send, it must be a bytestring as SecretBox will treat is as just a binary blob of data.
message = b"The president will be exiting through the lower levels"

# This is a nonce, it *MUST* only be used once, but it is not considered secret and can be transmitted or stored alongside
# the ciphertext. A good source of nonce is just 24 random bytes.
nonce = nacl.utils.random(nacl.secret.SecretBox.NONCE_SIZE)

# Encrypt our message, it will be exactly 40 bytes longer than the original message as it stores authentication 
# information and nonce alongside it.
encrypted = box.encrypt(message, nonce)

# Decrypt our message, an exception will be raised if the encryption was tampered with or there was otherwise an error.
plaintext = box.decrypt(encrypted)

In [ ]:
%reload_ext version_information
%version_information numpy, scipy, matplotlib, pycrypto, cryptography, nacl, streql, version_information